In [90]:
import pandas as pd
from math import *
import numpy as np
import datetime 

In [196]:
data = pd.read_csv('C:/Users/renz/Downloads/hour_interval_out.csv')

In [113]:
data = pd.read_csv('C:/Users/renz/Downloads/day_interval_out.csv')

In [114]:
port = pd.read_csv('C:/Users/renz/Downloads/1130/port.csv')

In [207]:
data1 = pd.read_csv('C:/Users/renz/Downloads/start_end_out.csv')

In [208]:
data1['ETA'] = data1['ETA'].apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

In [209]:
def distance(group):
    R = 6373.0
    lat1 = radians(group.reset_index().loc[0, 'LAT'])
    lon1 = radians(group.reset_index().loc[0, 'LON'])
    lat2 = radians(group.reset_index().loc[1, 'LAT'])
    lon2 = radians(group.reset_index().loc[1, 'LON'])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a)) 
    distance = R * c
    return distance

In [210]:
data_start_end = data1.groupby('MMSI2').agg({'ETA' : lambda x: str(x.iloc[1,] - x.iloc[0,])[:2]})

In [211]:
pd.merge(data_start_end, pd.DataFrame(data1.groupby('MMSI2').apply(distance)), how='left', on='MMSI2').to_csv('C:/Users/renz/Downloads/start_end2.csv')

# 6시간으로 끊기( MMSI3 제작 )

In [243]:
data = pd.read_csv('C:/Users/renz/Downloads/data_complete/hour_interval_out.csv')
data['ETA2'][pd.isnull(data['ETA2'])] = data['ETA'][pd.isnull(data['ETA2'])]
data['ETA'] = data['ETA'].apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
data['ETA2'] = data['ETA2'].apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [259]:
data['HOUR_DIFF'] = data['ETA']-data['ETA2']

In [299]:
data['HOUR_DIFF2'] = data['HOUR_DIFF'].apply(lambda x: str(x)[-8:-6])

In [321]:
L = []
for i in data['HOUR_DIFF2']>='06':
    if i == True:
        L.append(1)
    else:
        L.append(0)
data['HOUR_DIFF3'] = L

In [339]:
L = []
j = 1
k = data.loc[0,'MMSI2']
for i1, i2 in enumerate(data['HOUR_DIFF3']):
    if k != data.loc[i1,'MMSI2']:
        j = 1
        k = data.loc[i1,'MMSI2']
    else:
        j = j + i2
    L.append(j)
data['HOUR_DIFF4'] = L

In [360]:
L = []
for i,j in zip(data['MMSI2'], data['HOUR_DIFF4']):
    L.append(str(i) + '_' + str(j))
data['MMSI3'] = L

In [374]:
count10_data = pd.DataFrame(data.groupby('MMSI3')['MMSI'].count()[data.groupby('MMSI3')['MMSI'].count()>10]).index

In [378]:
data1 = data[data['MMSI3'].apply(lambda x: x in count10_data)]

In [421]:
data1.to_csv('C:/Users/renz/Downloads/data_complete/1204_hour_interval_out_MMSI3.csv', index=False)

In [398]:
data2 = pd.DataFrame(data1.groupby('MMSI3').first())
data2 = data2.reset_index()

In [399]:
data3 = pd.DataFrame(data1.groupby('MMSI3').last())
data3 = data3.reset_index()

In [406]:
data_concat = pd.concat([data2, data3])

In [409]:
port = pd.read_csv('C:/Users/renz/Downloads/1130/port.csv')

In [411]:
L2 = []
for i,j,k in zip(data_concat['MMSI3'], data_concat['LAT'], data_concat['LON']):
    L1 = []
    for l1, l2 in zip(port['LAT'], port['LON']):
        R = 6373.0 
        lat1 = radians(j) 
        lon1 = radians(k) 
        lat2 = radians(l1) 
        lon2 = radians(l2) 
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2 
        c = 2 * atan2(sqrt(a), sqrt(1 - a)) 
        distance = R * c
        L1.append(distance)
    L2.append(min(L1))

In [7]:
L2 = []
for i,j,k in zip(data1['MMSI2'], data1['LAT'], data1['LON']):
    L1 = []
    for l1, l2 in zip(data2['LATITUDE'], data2['LONGITUDE']):
        R = 6373.0 
        lat1 = radians(j) 
        lon1 = radians(k) 
        lat2 = radians(l1) 
        lon2 = radians(l2) 
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2 
        c = 2 * atan2(sqrt(a), sqrt(1 - a)) 
        distance = R * c
        L1.append(distance)
    L2.append(min(L1))

In [413]:
data_concat['distance'] = L2

In [414]:
data_5km = list(data_concat[data_concat['distance'] <= 5]['MMSI3'].value_counts()[data_concat[data_concat['distance'] <= 5]['MMSI3'].value_counts() == 2].index)

In [416]:
data3 = data_concat[data_concat['MMSI3'].apply(lambda x: x in data_5km)]

In [418]:
data3.to_csv('C:/Users/renz/Downloads/1204_start_end.csv', index=False)

In [8]:
data1['distance'] = L2

In [9]:
data_5km = list(data1[data1['distance'] <= 5]['MMSI2'].value_counts()[data1[data1['distance'] <= 5]['MMSI2'].value_counts() == 2].index)

In [10]:
data3 = data1[data1['MMSI2'].apply(lambda x: x in data_5km)]